## Install Libraries

In [26]:
# !pip install torchsummary
# !pip install torchgeometry
# !pip install segmentation-models-pytorch

## Import Libraries

In [27]:
import os
import pandas as pd
import numpy as np
import cv2
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, random_split, DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from torchvision.transforms import ToTensor
from PIL import Image
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
from torchvision import transforms
from torchinfo import summary
import timm
import segmentation_models_pytorch as smp
import wandb

In [28]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-08e26562-a64e-7070-3501-54e4b1759396)


In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Read data

In [30]:
images_path = "/kaggle/input/bkai-igh-neopolyp/train/train/"
image_path = []
TRAIN_DIR = '/kaggle/input/bkai-igh-neopolyp/train/train'
for root, dirs, files in os.walk(TRAIN_DIR):
    for file in files:
        path = os.path.join(root,file)
        image_path.append(path)
        
len(image_path)

1000

In [31]:
mask_path = []
TRAIN_MASK_DIR = '/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt'
for root, dirs, files in os.walk(TRAIN_MASK_DIR):
    for file in files:
        path = os.path.join(root,file)
        mask_path.append(path)
        
len(mask_path)

1000

In [32]:
class DatasetCustom(Dataset):
    def __init__(self, img_dir, label_dir, resize=None, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.resize = resize
        self.transform = transform
        self.images = os.listdir(self.img_dir)

    def __len__(self):
        return len(self.images)
    
    def read_mask(self, mask_path):
        image = cv2.imread(mask_path)
        image = cv2.resize(image, self.resize)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        lower_red1 = np.array([0, 100, 20])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([160,100,20])
        upper_red2 = np.array([179,255,255])
        
        lower_mask_red = cv2.inRange(image, lower_red1, upper_red1)
        upper_mask_red = cv2.inRange(image, lower_red2, upper_red2)
        
        red_mask = lower_mask_red + upper_mask_red
        red_mask[red_mask != 0] = 1

        green_mask = cv2.inRange(image, (36, 25, 25), (70, 255, 255))
        green_mask[green_mask != 0] = 2

        full_mask = cv2.bitwise_or(red_mask, green_mask)
        full_mask = np.expand_dims(full_mask, axis=-1) 
        full_mask = full_mask.astype(np.uint8)
        
        return full_mask

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        label_path = os.path.join(self.label_dir, self.images[idx])
        image = cv2.imread(img_path)  #  BGR
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert to RGB
        label = self.read_mask(label_path)  
        image = cv2.resize(image, self.resize)
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [33]:
img_resize = (256, 256)

In [34]:
dataset = DatasetCustom(img_dir= TRAIN_DIR,
                             label_dir= TRAIN_MASK_DIR,
                             resize= img_resize,
                             transform = None)

## Augmentation

In [35]:
class AugmentDataset(Dataset):
    def __init__(self, dataset, transform=None, length_multiplier=1):
        self.dataset = dataset
        self.transform = transform
        self.length_multiplier = length_multiplier
        
    def __getitem__(self, idx):
        actual_idx = idx % len(self.dataset)
        image, label = self.dataset[actual_idx]
        if self.transform:
            transformed = self.transform(image=image, mask=label)
            image = transformed['image'].float()
            label = transformed['mask'].float()
            label = label.permute(2, 0, 1)
        return image, label

    def __len__(self):
        return len(self.dataset) * self.length_multiplier

In [36]:
train_transformation = A.Compose([
    A.HorizontalFlip(p=0.4),
    A.VerticalFlip(p=0.4),
    A.RandomGamma (gamma_limit=(70, 130), eps=None, always_apply=False, p=0.2),
    A.RGBShift(p=0.3, r_shift_limit=10, g_shift_limit=10, b_shift_limit=10),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

val_transformation = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

In [37]:
# Define the split sizes
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Define the batch size
batch_size = 32
# Randomly split the dataset
generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=generator)

train_dataset = AugmentDataset(train_dataset, transform=train_transformation, length_multiplier=3)
val_dataset = AugmentDataset(val_dataset, transform=val_transformation)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

## Define Model

In [38]:
import segmentation_models_pytorch as smp

model = smp.UnetPlusPlus(
    encoder_name="resnet50",        
    encoder_weights="imagenet",     
    in_channels=3,                  
    classes=3     
)

In [51]:
learning_rate = 0.0001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [40]:
color_dict= {0: (0, 0, 0),
             1: (255, 0, 0),
             2: (0, 255, 0)}
def mask_to_rgb(mask, color_dict):
    output = np.zeros((mask.shape[0], mask.shape[1], 3))

    for k in color_dict.keys():
        output[mask==k] = color_dict[k]

    return np.uint8(output)    

In [41]:
del dataset
del train_dataset
del val_dataset

## Train

In [42]:
wandb.login(
    # set the wandb project where this run will be logged
#     project= "PolypSegment", 
    key = "e02f7703b40a2b3e0ab4801d4cb1d86b3b7327a6",
)
wandb.init(
    project = "PolypSegment"
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Train_loss,█▃▂▁▁▁▁▁▁
Val_loss,█▃▂▂▁▁▂▂▂
Train_loss,0.0114
Val_loss,0.0642


In [52]:
from tqdm import tqdm

num_epochs = 5

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
best_val_loss = 999

epoch_bar = tqdm(total=num_epochs, desc='Total Progress')

for epoch in range(20, 20 + num_epochs):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        labels = labels.squeeze(dim=1).long()
        outputs = model(images)
    
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
        
#         del images, labels, outputs, loss
#         torch.cuda.empty_cache()
        
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            labels = labels.squeeze(dim=1).long()
            
            outputs = model(images)

            val_loss += criterion(outputs.float(),labels.long()).item()
            
#             del images, labels, outputs
#             torch.cuda.empty_cache()
            
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {val_loss/len(val_loader):.10f}")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        checkpoint = { 
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'loss': val_loss,
        }
        save_path = f'model.pth'
        torch.save(checkpoint, save_path)
        
    epoch_bar.update(1)
    wandb.log({'Val_loss': val_loss/len(val_loader),'Train_loss': train_loss/len(train_loader)})
epoch_bar.close()


Total Progress:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch [21/5], Loss: 0.0716063439



Total Progress:  20%|██        | 1/5 [02:34<10:16, 154.13s/it]

Epoch [22/5], Loss: 0.0661033477



Total Progress:  40%|████      | 2/5 [05:06<07:39, 153.09s/it]

Epoch [23/5], Loss: 0.0608807829



Total Progress:  80%|████████  | 4/5 [10:10<02:32, 152.36s/it]

Epoch [24/5], Loss: 0.0775313045



Total Progress: 100%|██████████| 5/5 [12:42<00:00, 152.40s/it]

Epoch [25/5], Loss: 0.0664264826


In [44]:
# model = smp.UnetPlusPlus(
#     encoder_name="resnet34",        
#     encoder_weights="imagenet",     
#     in_channels=3,                  
#     classes=3     
# )
# checkpoint = torch.load('/kaggle/input/unet/pytorch/default/1/model.pth')
# model.load_state_dict(checkpoint['model'])
# device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
# model.to(device)

In [45]:
!mkdir prediction

mkdir: cannot create directory ‘prediction’: File exists


## Get Testset Prediction

In [53]:
model.eval()
for i in os.listdir("/kaggle/input/bkai-igh-neopolyp/test/test"):
    img_path = os.path.join("/kaggle/input/bkai-igh-neopolyp/test/test", i)
    ori_img = cv2.imread(img_path)
    ori_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)
    ori_h = ori_img.shape[0]
    ori_w = ori_img.shape[1]
    img = cv2.resize(ori_img, img_resize, interpolation=cv2.INTER_AREA)
    transformed = val_transformation(image=img)
    input_img = transformed["image"]
    input_img = input_img.unsqueeze(0).to(device)
    with torch.no_grad():
        output_mask = model(input_img).squeeze(0).cpu().numpy().transpose(1, 2, 0)
    mask = cv2.resize(output_mask, (ori_w, ori_h), interpolation=cv2.INTER_CUBIC)
    mask = np.argmax(mask, axis=2)
    mask_rgb = mask_to_rgb(mask, color_dict)
    mask_rgb = cv2.cvtColor(mask_rgb, cv2.COLOR_RGB2BGR)
    cv2.imwrite("prediction/{}".format(i), mask_rgb)
    # Clear variables to free memory
    del img, input_img, output_mask, mask, mask_rgb
    torch.cuda.empty_cache()

In [47]:
def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 225] = 255
    pixels[pixels <= 225] = 0
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    
    return rle_to_string(rle)

def rle2mask(mask_rle, shape=(3,3)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def mask2string(dir):
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    for image_id in os.listdir(dir):
        id = image_id.split('.')[0]
        path = os.path.join(dir, image_id)
        print(path)
        img = cv2.imread(path)[:,:,::-1]
        h, w = img.shape[0], img.shape[1]
        for channel in range(2):
            ws.append(w)
            hs.append(h)
            ids.append(f'{id}_{channel}')
            string = rle_encode_one_mask(img[:,:,channel])
            strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r


MASK_DIR_PATH = '/kaggle/working/prediction'
dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']

df.to_csv(r'output.csv', index=False)

/kaggle/working/prediction/f62f215f0da4ad3a7ab8df9da7386835.jpeg
/kaggle/working/prediction/a6a4248a41e8db8b4ed633b456aaafac.jpeg
/kaggle/working/prediction/4417fda8019410b1fcf0625f608b4ce9.jpeg
/kaggle/working/prediction/7cb2eb1ef57af2ed9fbb63b28163a745.jpeg
/kaggle/working/prediction/cb1b387133b51209db6dcdda5cc8a788.jpeg
/kaggle/working/prediction/cf464aa36bf7c09a3bb0e5ca159410b9.jpeg
/kaggle/working/prediction/aeeb2b535797395305af926a6f23c5d6.jpeg
/kaggle/working/prediction/a9d45c3dbc695325ded465efde988dfb.jpeg
/kaggle/working/prediction/7af2ed9fbb63b28163a745959c039830.jpeg
/kaggle/working/prediction/5c1346e62522325c1b9c4fc9cbe1eca1.jpeg
/kaggle/working/prediction/cc5cfd263f1f90be28799235026b3550.jpeg
/kaggle/working/prediction/1db239dda50f954ba59c7de13a35276a.jpeg
/kaggle/working/prediction/dc0bb223c4eaf3372eae567c94ea04c6.jpeg
/kaggle/working/prediction/0a5f3601ad4f13ccf1f4b331a412fc44.jpeg
/kaggle/working/prediction/e7998934d417cb2eb1ef57af2ed9fbb6.jpeg
/kaggle/working/predictio

## Inference

In [48]:
# from torch.jit import load
# model = UNet()
# optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

# checkpoint = torch.load(pretrained_path)

In [49]:
# optimizer.load_state_dict(checkpoint['optimizer'])

In [50]:
# from collections import OrderedDict
# new_state_dict = OrderedDict()
# for k, v in checkpoint['model'].items():
#     name = k[7:] # remove `module.`
#     new_state_dict[name] = v
# # load params
# model.load_state_dict(new_state_dict)